In [1]:
%matplotlib qt
import os
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm
from mne.io import read_raw_brainvision, read_raw_fif

from scipy.signal import find_peaks
import numpy as np
from mne.channels import read_dig_captrak, make_standard_montage
from mne.preprocessing import ICA
from mne_icalabel import label_components
import customtkinter as ctk
from mne.coreg import Coregistration
from mne import (open_report, events_from_annotations, Epochs, write_trans,
                setup_source_space, make_bem_model, make_bem_solution, make_forward_solution, compute_covariance,
                extract_label_time_course, read_labels_from_annot)
from mne.minimum_norm import make_inverse_operator, apply_inverse
from mne.datasets import fetch_fsaverage
from scipy.signal import find_peaks
import warnings
from mne.preprocessing import ICA, EOGRegression, create_eog_epochs, create_ecg_epochs, compute_proj_ecg, compute_proj_eog, EOGRegression

In [2]:
%%latex
##### for preprocessing()
## ask philip where is PO7, PO8, O1 and O2 and adjust it here
## put these cahnnels as misc and see what we can do with it
## kernel crash when running ctk
## ssp for ecg (pulse)
## ssp for vertical eye
## regression for horizontal eye
## .load_data()
## add teas
## GSR, X, Y, Y drop
## rename y-axis in pulse plot maybe *1e9 and time shows
## mne.preprocessing.annotate_break
## interpolate bads (mne.preprocessing.find_bad_channels_lof)



##### for run_analysis()
## in gpias dont do baselining
## double check with repo codes
## ask gpias triggers code
## documenatiaion
## 

<IPython.core.display.Latex object>

In [3]:
x = "/Volumes/Extreme SSD/payam_data/antinomics_data"
from pathlib import Path
x= Path(x)

In [4]:
from files_preparation import create_subjects_dir

subject_id = "xrtt"
create_subjects_dir(subject_id=subject_id,
                        #raws_dir="/Volumes/G_USZ_ORL$/Research/ANTINOMICS/data",
                        raws_dir="/home/toedtli/Tinnitus/Data/tide/project/",
                        subjects_dir="/tmp/test/",
                        skip_list=["gpias", "xxxxx", "xxxxy", "omi"])

/home/toedtli/Tinnitus/Antinomics_Github/files_preparation.py:62: UserWarning: Expected to see a directory /tmp/test
  warnings.warn(f"Expected to see a directory {subjects_dir}", UserWarning)
/home/toedtli/Tinnitus/Antinomics_Github/files_preparation.py:63: UserWarning: Subject xrtt not found in the E directory!
  warnings.warn(f"Subject {subject_id} not found in the {title} directory!", UserWarning)


In [5]:
raise NotImplementedError()

NotImplementedError: 

In [5]:
%matplotlib qt
from eeg.eeg_preprocessing import preprocessing
preprocessing(subject_id="pat26",
                subjects_dir="/tmp/test",#"/Users/payamsadeghishabestari/antinomics_clean_codes/subjects",
                paradigm="rest",
                manual_data_scroll=True,
                run_ica=False,
                manual_ica_removal=False,
                eog_correct=False,
                pulse_correct=False,
                resp_correct=False,
                create_report=True,
                saving_dir=None,
                overwrite=True,
                verbose="ERROR")

 20%|█████████                                    

Loading raw EEG data ...



 40%|██████████████████                           

DigMontage is only a subset of info. There are 2 channel positions not present in the DigMontage. The channels missing from the montage are:

['HRli', 'HRre'].

Consider using inst.rename_channels to match the montage nomenclature, or inst.set_channel_types if these are not EEG channels, or use the on_missing parameter if the channel positions are allowed to be unknown in your analyses.
dropping channels
Resampling, filtering and re-referencing ...



qt.core.qobject.connect: QObject::connect(QStyleHints, QStyleHints): unique connections require a pointer to member function of a QObject subclass
/home/toedtli/Tinnitus/Antinomics_Github/.venv/lib/python3.10/site-packages/mne_qt_browser/_pg_figure.py:3061: RuntimeWarning: Failed to disconnect (None) from signal "triggered()".
  sig.disconnect()
 80%|████████████████████████████████████         

Creating report and saving...

EEG data were preprocessed sucessfully!



In [6]:
%matplotlib qt
from eeg.eeg_processing import run_rs_analysis

run_rs_analysis(
        subject_id="pat26",
        subjects_dir="/tmp/test",
        event_ids=None,
        source_analysis=True,
        mri=False,
        subjects_fs_dir=None,
        manual_data_scroll=False,
        create_report=True,
        saving_dir=None,
		overwrite=True,
        verbose="ERROR"
        )


 12%|█████▋                                       

Loading preprocessed EEG data ...

Creating epochs...

Warning- please check eyes open and eyes closed events are assumptions!


 88%|███████████████████████████████████████▍     

Creating report...

True
True


100%|█████████████████████████████████████████████

Analysis finished successfully!



In [7]:
si='asf'+'ff'

In [8]:
%matplotlib qt
from eeg.eeg_processing import run_erp_analysis

run_erp_analysis(
        subject_id="pat26",
        subjects_dir="/tmp/test",
        paradigm="rest",
        events=None,
        source_analysis=True,
        mri=False,
        subjects_fs_dir=None,
        manual_data_scroll=False,
        create_report=True,
        saving_dir=None,
        verbose="ERROR"
        )

 11%|█████                                        

Loading preprocessed EEG data ...



NotImplementedError: 

Eyetracking

In [2]:
import mne
raw = mne.io.read_raw_brainvision("/Users/payamsadeghishabestari/antinomics_clean_codes/subjects/zdfy/EEG/xxxxx/zdfy_xxxxx.vhdr")
mne.read_events(raw)

In [2]:
raw.annotations

In [1]:
from mne.io import read_raw_brainvision, read_raw_eyelink
from mne import events_from_annotations
from mne.preprocessing import realign_raw


## load
fname_eeg = "/Users/payamsadeghishabestari/Downloads/alertness_01_H036.vhdr"
fname_et = "/Users/payamsadeghishabestari/Downloads/local_al1H036.asc"

raw_eeg = read_raw_brainvision(fname_eeg, preload=True)
raw_eye = read_raw_eyelink(fname=fname_et, create_annotations=True) 
events_eeg, event_ids_eeg = events_from_annotations(raw_eeg)
events_eye, event_ids_eye = events_from_annotations(raw_eye)

## for now
stim_id_eeg_1, stim_id_eeg_2 = 8, 9
stim_id_eye_1, stim_id_eye_2 = 4, 5

s_raw = events_eeg[(events_eeg[:,2] == stim_id_eeg_1) | (events_eeg[:,2] == stim_id_eeg_2)][:, 0]
s_other = events_eye[(events_eye[:,2] == stim_id_eye_1) | (events_eye[:,2] == stim_id_eye_2)][:, 0]

## realigning
realign_raw(raw=raw_eeg,
            other=raw_eye,
            t_raw=s_raw / raw_eeg.info["sfreq"] - raw_eeg.first_time,
            t_other=s_other / raw_eye.info["sfreq"] - raw_eye.first_time,
            verbose=None)

raw_eye.add_channels([raw_eeg], force_update_info=True)
events, event_dict = events_from_annotations(raw_eye)
del raw_eeg  

In [1]:
fname_eeg = "/Users/payamsadeghishabestari/Downloads/alertness_01_H036.vhdr"
fname_et = "/Users/payamsadeghishabestari/Downloads/local_al1H036.asc"

raw_eeg = read_raw_brainvision(fname_eeg, preload=True)
raw_eye = read_raw_eyelink(fname=fname_et, create_annotations=True) 

In [1]:
raw_eye.plot()

In [1]:
raw_eye.plot()